<h1 style='color:blue'> PRUEBA FINAL DE PYTHON </h1>

In [1]:
# Esto recarga automáticamente los módulos cuando se cambien ya que Jupyter cachea los módulos importados. 
# Si se modifica el archivo después de importarlo, se necesita recargar y para no reiniciar el kernel del jupyter

# %load_ext autoreload
# %autoreload 2

#param_db = {
#    'HOST': 'localhost',
#    'USERNAME': 'postgres',
#    'PASSWORD': 12345,
#    'DATABASE' : 'g108_classicmodels',
#    'PORT': 5432
#}

from funciones.funcionesPrueba import create_DB,acond_DB
#CREO LA BD DE LA PRUEBA
create_DB()
#ACONDICIONO LA BD DE LA PRUEBA
acond_DB()


Base de datos g108_classicmodels ya existe. Usando existente.
Limpiando tablas existentes...
Tabla orderdetails eliminada (si existía)
Tabla orders eliminada (si existía)
Tabla customers eliminada (si existía)
Tabla employees eliminada (si existía)
Tabla products eliminada (si existía)
Tabla top_10_clientes_2005 eliminada (si existía)
Tabla top_10_productos_2005 eliminada (si existía)
Tabla offices eliminada (si existía)
Tabla payments eliminada (si existía)
Tabla productlines eliminada (si existía)
Importando archivo SQL...
Archivo classicmodels_insert.sql ejecutado correctamente


<h1 style='color:green' > Descripción </h1>

El área comercial de una empresa pide realizar un cierre de año de las ventas, tanto para revisar si las metas fueron cumplidas, como para poder planiﬁcar el siguiente año. Para ello, considerarán los datos del dataset classicmodels.sql para responder algunas preguntas, realizando las siguientes tareas.

Ejercicio 1. 
Genera una función llamada leer_tabla(tabla, engine)y utilízala para leer tablas completas desde la base de datos en dataframes independientes. Utilizando esta función, importa las siguientes tablas:
- order
- orderdetails 
- customers
- products
- employees

In [3]:

from funciones.funcionesPrueba import leer_tabla
import pandas as pd

# engine = create_engine(f"postgresql://{param_db['USERNAME']}:{param_db['PASSWORD']}@{param_db['HOST']}/{param_db['DATABASE']}")
# revision = inspect(engine)
# tablas = revision.get_table_names()
# tablas
# tablas_varias = {}
#for t in tablas:
#    query = f"select * from {t};"
#    tablas_varias[t]= pd.read_sql(query, engine)
#tablas_varias

#def leer_tabla(tabla,eng):
#    query = f"select * from {tabla};"
#    df = pd.read_sql(query, eng)
#    return df
tablas = ['orders','orderdetails','customers','products','employees']
orders_df = leer_tabla (tablas[0])
orderdetails_df = leer_tabla (tablas[1])
customers_df = leer_tabla (tablas[2])
products_df = leer_tabla (tablas[3])
employees_df = leer_tabla (tablas[4])
orders_df.shape
# orders_df['status'].unique()

(326, 7)

Ejercicio 2.
Realiza el cruce entre los DataFrames, asegurándote de utilizar correctamente el parámetro validate para asegurar la integridad referencial.

In [5]:
df_base = (orders_df
    .merge(orderdetails_df, on="orderNumber", how="left",validate="one_to_many")
    .merge(customers_df, on="customerNumber", how="left",validate="many_to_one")
    .merge(products_df, on="productCode", how="left",validate="many_to_one")
    .merge(employees_df, left_on='salesRepEmployeeNumber',right_on='employeeNumber', how="left",validate="many_to_one"))

Ejercicio 3. Agrega las siguientes columnas, considerando su nombre y la fórmula asociada
- venta: quantityOrdered*priceEach
- costo: quantityOrdered*buyPrice
- ganancia: considerando las columnas anteriores

In [7]:
df_base['venta'] = df_base['quantityOrdered']*df_base['priceEach']
df_base['costo'] = df_base['quantityOrdered']*df_base['buyPrice']
df_base['ganancia']=df_base['venta']-df_base['costo']

Ejercicio 4. ¿Cuál fue el total de ventas por línea de productos? Incluye una ﬁla de totales.

In [11]:
df_base.head(20)
df_base_pivot_table = pd.pivot_table(data = df_base
                                    , index=  "productLine"
                                    , values = "quantityOrdered"
                                    , aggfunc ="sum"
                                    , margins=True
                                    , margins_name='Total')
df_base_pivot_table

,quantityOrdered
productLine,
Classic Cars,35582
Motorcycles,12778
Planes,11872
Ships,8532
Trains,2818
Trucks and Buses,11001
Vintage Cars,22933
Total,105516


Ejercicio 5. ¿Cuántos clientes distintos hicieron compras?

In [13]:
#df_base.head(5)

# df_base['status'].unique()
# df_base['customerNumber'].unique().size
# clientes_disntintos_compras = df_base['customerNumber'][df_base['status']=='Resolved'].unique()
print(f"Hicieron compras {df_base['customerNumber'].unique().size} clientes distintos")

Hicieron compras 98 clientes distintos


Ejercicio 6.¿Existen clientes que aún no han hecho ninguna compra? ¿Cuántos son?

In [15]:
clientes_totales = customers_df['customerNumber'].unique().size
clientes_compras = df_base['customerNumber'].unique().size
print (f"Existen clientes que no han hecho ninguna compra y son {clientes_totales - clientes_compras} clientes")

Existen clientes que no han hecho ninguna compra y son 24 clientes


Ejercicio 7.  Se solicita la creación de dos reportes, que respondan las preguntas dadas.



In [17]:
# REPORTE 1: CLIENTES QUE HAN REALIZADO COMPRAS (PREGUNTA 5)
clientes_con_compras = df_base.groupby(['customerNumber', 'customerName']).agg({
    'orderNumber': 'nunique',  
    'venta': 'sum',           
    'ganancia': 'sum'        
}).reset_index()

clientes_con_compras.columns = ['customerNumber', 'customerName', 'total_ordenes', 'total_ventas', 'total_ganancia']
clientes_con_compras['total_ventas'] = clientes_con_compras['total_ventas'].round(2)
clientes_con_compras['total_ganancia'] = clientes_con_compras['total_ganancia'].round(2)
clientes_con_compras = clientes_con_compras.sort_values('total_ventas', ascending=False)

print(f" Total de clientes que han realizado compras: {len(clientes_con_compras)} clientes")
print(f" Total de ventas generadas: ${clientes_con_compras['total_ventas'].sum():,.2f}")


# =========================================================================================================================================
# REPORTE 2: CLIENTES SIN COMPRAS (PREGUNTA 6)
todos_los_clientes = customers_df[['customerNumber', 'customerName', 'city', 'country']].copy()

clientes_que_compraron = df_base['customerNumber'].unique()

clientes_sin_compras = todos_los_clientes[
    ~todos_los_clientes['customerNumber'].isin(clientes_que_compraron)
].copy()

clientes_sin_compras['estado'] = 'Sin compras'
clientes_sin_compras['total_ventas'] = 0.0
clientes_sin_compras['total_ordenes'] = 0

clientes_sin_compras = clientes_sin_compras[['customerNumber', 'customerName', 'city', 'country', 'estado', 'total_ordenes', 'total_ventas']]

print(f" Total de clientes registrados: {len(todos_los_clientes)} clientes")
print(f" Clientes que han comprado: {len(clientes_que_compraron)} clientes")
print(f" Clientes sin compras: {len(clientes_sin_compras)} clientes")


 Total de clientes que han realizado compras: 98 clientes
 Total de ventas generadas: $9,604,190.61
 Total de clientes registrados: 122 clientes
 Clientes que han comprado: 98 clientes
 Clientes sin compras: 24 clientes


- ¿Cuáles fueron los 10 clientes que reportan mayores ventas brutas en dinero durante el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada top_10_clientes_2005, en la que se especiﬁque el nombre del cliente y su correspondiente venta, costo y ganancia.

In [19]:
from funciones.funcionesPrueba import escribir_BD,filt_DF_fechas,generar_reporte_pivotado
# df_base.head(20)

# Filtrar por año
tabla_2005 = filt_DF_fechas(df_base, 'orderDate', 'año', año = 2005)

tabla_2005_pivot_table_clientes= generar_reporte_pivotado(
    df=tabla_2005,
    filas='customerName',
    valores=['venta', 'costo', 'ganancia'],
    funcion_agrupadora='sum'
)
#tabla_2005_pivot_table_clientes = pd.pivot_table(
#    data=tabla_2005,
#    index='customerName', 
#    values=['venta', 'costo', 'ganancia'],
#    aggfunc='sum'
#)
tabla_final_clientes = tabla_2005_pivot_table_clientes.sort_values(by=['venta'],ascending=False).iloc[0:10,:]
escribir_BD(tabla_final_clientes,'top_10_clientes_2005')
# df_base.head(5)

Empezando a escribir la tabla top_10_clientes_2005
Terminando de escribir la tabla top_10_clientes_2005


- ¿Cuál fue el top 10 de artículos más vendidos durante el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada top_10_productos_2005, en la que se especiﬁque el nombre del producto y su correspondiente venta, costo y ganancia.

In [21]:
#df_base["orderDate"] = pd.to_datetime(df_base["orderDate"], format="%Y-%m-%d")
#df_base["requiredDate"] = pd.to_datetime(df_base["requiredDate"], format="%Y-%m-%d")
#df_base["shippedDate"] = pd.to_datetime(df_base["shippedDate"], format="%Y-%m-%d")
# tabla_2005 = df_base[df_base['requiredDate'].dt.year == 2005]
# productLine
tabla_2005 = filt_DF_fechas(df_base, 'orderDate', 'año', año = 2005)
tabla_2005_pivot_table_productos = generar_reporte_pivotado(
    df=tabla_2005,
    filas='productName',
    valores=['quantityOrdered','venta','costo','ganancia'],
    funcion_agrupadora='sum'
)
tabla_final_productos = tabla_2005_pivot_table_productos.sort_values(by=['quantityOrdered'],ascending=False).iloc[0:10,:]
escribir_BD(tabla_final_productos,'top_10_productos_2005')

Empezando a escribir la tabla top_10_productos_2005
Terminando de escribir la tabla top_10_productos_2005
